In [1]:
import json
import boto3
from typing import Dict

SECRET_NAME = "ds4a-project-dev-db-connection-secret"


def get_secret(aws: boto3.Session, secret_name: str) -> Dict[str, str]:
    sm = aws.client("secretsmanager")
    response = sm.get_secret_value(SecretId=secret_name)
    secret = response["SecretString"]
    return json.loads(secret)


aws = boto3.Session()
params = get_secret(aws, SECRET_NAME)
params.keys()


dict_keys(['password', 'dbname', 'engine', 'port', 'dbInstanceIdentifier', 'host', 'username'])

In [2]:
import psycopg2


conn = psycopg2.connect(
    host=params["host"],
    database=params["dbname"],
    user=params["username"],
    password=params["password"],
)

cur = conn.cursor()
cur.execute("SELECT version()")
cur.fetchone()


('PostgreSQL 13.4 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 7.3.1 20180712 (Red Hat 7.3.1-6), 64-bit',)

In [8]:
from psycopg2.extras import RealDictCursor

query = """--sql
    SELECT 
        m.name as municipality_name,
        m.code as municipality_code,
        d.name as department_name,
        d.code as department_code,
        mph.total as population,
        sum(sa.count) as suicide_attempts,
        CASE
            WHEN m.latitude = 'NaN' THEN NULL
            ELSE m.latitude
        END AS latitude,
        CASE
            WHEN m.longitude = 'NaN' THEN NULL
            ELSE m.longitude
        END AS longitude
    FROM suicideAttempts sa
    JOIN municipality m
        ON sa.municipality_id = m.code
    JOIN departments d
        ON m.department_id = d.code
    JOIN municipalityPopulationHistory mph 
        ON sa.municipality_id = mph.municipality_id AND sa.year = mph.year
    WHERE sa.year = %s
    GROUP BY
        municipality_name,
        municipality_code,
        department_name,
        department_code,
        population
    """

values = [2020]

try:
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute(query, values)
    res = cursor.fetchall()
    conn.commit()
except Exception as e:
    conn.rollback()
    print(e)

resultset = [dict(row) for row in res]
print(f"{len(resultset)=:,d}")
resultset[:10]


syntax error at or near "mph"
LINE 7:         mph.total as population,
                ^

len(resultset)=460


[{'municipality_name': 'Medellín',
  'municipality_code': 5001,
  'department_name': 'Antioquia',
  'department_code': 5,
  'population': 2533424,
  'suicides': 173,
  'latitude': Decimal('6.24858636743'),
  'longitude': Decimal('-75.5760017134')},
 {'municipality_name': 'Abejorral',
  'municipality_code': 5002,
  'department_name': 'Antioquia',
  'department_code': 5,
  'population': 20287,
  'suicides': 1,
  'latitude': Decimal('5.78930111433'),
  'longitude': Decimal('-75.4287403904')},
 {'municipality_name': 'Amagá',
  'municipality_code': 5030,
  'department_name': 'Antioquia',
  'department_code': 5,
  'population': 31283,
  'suicides': 1,
  'latitude': Decimal('6.03880842381'),
  'longitude': Decimal('-75.7019483389')},
 {'municipality_name': 'Amalfi',
  'municipality_code': 5031,
  'department_name': 'Antioquia',
  'department_code': 5,
  'population': 27071,
  'suicides': 1,
  'latitude': Decimal('6.90967360478'),
  'longitude': Decimal('-75.0774749239')},
 {'municipality_name

In [51]:
delete_statement = """--sql
DELETE FROM
    interfamilyViolence a
        USING interfamilyViolence b
WHERE
    a.id > b.id
    AND a.year = b.year
    AND a.count = b.count
    AND a.municipality_id = b.municipality_id;
"""

try:
    cur.execute(delete_statement)
    conn.commit()
except Exception as e:
    print(e)
    conn.rollback()


In [6]:
from psycopg2.extras import RealDictCursor

query = """--sql
SELECT table_name FROM information_schema.tables
    WHERE table_schema = 'public'
"""

try:
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute(query)
    res = cursor.fetchall()
    conn.commit()
except:
    res = None
    conn.rollback()

res


[RealDictRow([('table_name', 'departments')]),
 RealDictRow([('table_name', 'departmentspopulationhistory')])]

In [8]:
conn.close()